# Imports

In [ ]:
import pickle
import yaml
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

%matplotlib inline

In [ ]:
with open('ros_packages/px4_ros_extended/src_py/params.yaml') as info:
    info_dict = yaml.load(info, Loader=yaml.SafeLoader)
        
n_test = info_dict["test_window_reward"]
n_train = info_dict["train_window_reward"]

# Dataframe Creation

In [ ]:
with open("logs/log_3.pkl", "rb") as file_pkl:
    data = pickle.load(file_pkl)
train = data[0]
test = data[1]

# Visualization

In [ ]:
# Visualize the result
means = pd.Series(train).rolling(n_train, min_periods=10).mean()
stds = pd.Series(train).rolling(n_train, min_periods=10).std()
x = range(0, len(means))
figure(figsize=(20, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);
plt.xticks(ticks=range(0, len(means), 100));

In [ ]:
means = pd.Series(test).rolling(n_test, min_periods=1).mean()
stds = pd.Series(test).rolling(n_test, min_periods=1).std()
x = range(0, len(means))
figure(figsize=(20, 8), dpi=80)
plt.plot(x, means, 'or')
plt.plot(x, means, '-', color='gray')

plt.fill_between(x, means - stds, means + stds,
                 color='gray', alpha=0.2)
plt.xlim(0, len(means)+1);
plt.xticks(ticks=range(0, len(means), 9), labels=range(500, len(train), 500));